### PromptTemplate 
* [PromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.prompt.PromptTemplate.html#langchain_core.prompts.prompt.PromptTemplate)
* [ChatPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatPromptTemplate.html#langchain_core.prompts.chat.ChatPromptTemplate)
* [ChatMessagePromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.chat.ChatMessagePromptTemplate.html#langchain_core.prompts.chat.ChatMessagePromptTemplate)
* [FewShotPromptTemplate](https://python.langchain.com/api_reference/core/prompts/langchain_core.prompts.few_shot.FewShotPromptTemplate.html#langchain_core.prompts.few_shot.FewShotPromptTemplate)
* PartialPrompt

In [2]:
# poetry add python-dotenv langchain langchain-openai

In [3]:
from dotenv import load_dotenv
import os
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:5])

gsk_3


##### 1) PromptTemplate 의 from_template() 함수 사용
* 주로 LLM(텍스트 완성형 모델, ex. Ollama, GPT-3.5)과 함께 사용
* 하나의 문자열 프롬프트를 생성

In [4]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from pprint import pprint

template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    #model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    #model="openai/gpt-oss-120b",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt_template | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3})
pprint(response)

('ChatGPT는 인터넷의 방대한 텍스트에서 다음 단어를 맞히는 예측 게임을 반복하며 패턴을 학습합니다.  \n'
 '이 과정에서 인간의 피드백(RLHF)으로 긍정적인 답변 순위를 조정해, 단순한 글 이어붙이기를 ‘도움이 되는 대화’로 재정렬합니다.  \n'
 '결국 통계적 패턴과 인간의 가치를 결합해, 주어진 맥락에서 자연스럽고 유용한 문장을 생성할 확률을 높이는 방식으로 작동합니다.')


##### 2) PromptTemplate 결합하기
* 동일한 Prompt 패턴을 사용하지만 여러 개의 질문을 작성해서 LLM을 실행할 수도 있습니다.

In [5]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(model_name="ChatGPT", count=3)

# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요")
              + "\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요."
)
combined_prompt.format(model_name="ChatGPT", count=3, language="한국어")
print(combined_prompt)

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
chain = combined_prompt | llm | StrOutputParser()
response = chain.invoke({"model_name":"ChatGPT", "count":3, "language":"한국어"})

pprint(response)

input_variables=['count', 'language', 'model_name'] input_types={} partial_variables={} template='{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요.\n\n 그리고 {model_name} 모델의 장점을 요약 정리해 주세요\n\n {model_name} 모델과 비슷한 AI 모델은 어떤 것이 있나요? 모델명은 {language}로 답변해 주세요.'
('**학습 원리 (3문장, 한국어)**  \n'
 'ChatGPT는 인터넷의 방대한 텍스트를 학습하여 단어가 등장할 확률을 익히고, 사람이 제공한 프롬프트에 맞춰 가장 적절한 다음 단어를 '
 '예측하며 순차적으로 문장을 생성합니다.  \n'
 '지도학습과 강화학습을 결합해 인간이 선호하는 답변을 보상으로 삼아 모델 스스로 답변 품질을 끌어올립니다.  \n'
 '결국 확률 계산과 보상 최대화를 반복하면서 대화 맥락을 파악하고 유창한 한국어를 포함한 다양한 언어로 응답할 수 있게 됩니다.\n'
 '\n'
 '**ChatGPT 모델의 주요 장점**  \n'
 '- 문맥을 길게 기억해 자연스럽고 연결된 대화 가능  \n'
 '- 코드 작성, 번역, 요약, 브레인스토밍 등 다양한 업무에 즉시 활용 가능  \n'
 '- 별도 학습 없이 프롬프트만으로 맞춤형 답변 생성  \n'
 '- 인터페이스가 간단해 코딩 지식 없이도 누구나 즉시 사용 가능  \n'
 '- 지속적인 업데이트로 최신 지식·보안·윤리 기준 반영\n'
 '\n'
 '**ChatGPT와 유사한 AI 모델(한국어 이름)**  \n'
 '1. 구글 젬마(Gemma)  \n'
 '2. 구글 젬바이니(Gemini, 구 버전은 바드)  \n'
 '3. 메타 라마3(Llama 3)  \n'
 '4. 클라우드 Claude(클로드)  \n'
 '5. MS 코파일럿(Copilot, 챗봇 버전)')


#### PromptTemplate 의 파라미터를 배열 형태로 하여 여러개 사용하는 경우

In [6]:
template_text = "{model_name} 모델의 학습 원리를 {count} 문장으로 한국어로 답변해 주세요."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

questions = [
    {"model_name": "GPT-4", "count": 3},
    {"model_name": "Gemini", "count": 4},
    {"model_name": "Claude", "count": 4}
]

# 여러 개의 프롬프트를 미리 생성
formatted_prompts = [prompt_template.format(**q) for q in questions] #파이썬에서 파라미터로 딕셔너리 받을 때 아스타(*) 2개 씀
print(formatted_prompts)  # 미리 생성된 질문 목록 확인

#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

for prompt in formatted_prompts:
    print(type(prompt), prompt)
    response = llm.invoke(prompt)
    pprint(response.content)

['GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.', 'Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.', 'Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.']
<class 'str'> GPT-4 모델의 학습 원리를 3 문장으로 한국어로 답변해 주세요.
('GPT-4는 수많은 텍스트 데이터에서 다음에 올 단어를 예측하도록 사전학습되고, 사람이 제공한 정답과 비교해 미세 조정되는 방식으로 '
 '학습됩니다. 이 과정에서 손실 함수를 최소화하며 수십 억 개의 가중치가 반복적으로 갱신됩니다. 결과적으로 패턴과 의미를 통합해 새로운 '
 '입력에 대해 자연스러운 응답을 생성할 수 있게 됩니다.')
<class 'str'> Gemini 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Gemini는 텍스트·이미지·오디오·코드 등 다양한 데이터를 동시에 처리하는 멀티모달 구조로, 한 번에 여러 유형의 정보를 통합해 '
 '학습합니다.  \n'
 '대규모 Transformer 기반 네트워크에 MoE(Mixture-of-Experts) 기법을 적용해 필요한 전문 서브네트워크만 '
 '활성화하면서 효율적인 확장 학습을 실현합니다.  \n'
 '강화학습과 인간 피드백(RLHF)으로 생성 결과의 정확성과 안전성을 지속 보정하며, 사고 사슬(Chain-of-Thought) 추론 '
 '능력을 강화합니다.  \n'
 '마지막으로 대용량 토큰을 한 번에 처리하는 긴 맥락 창과 고급 정밀도(예: 32-bit→8-bit) 압춙 기술을 결합해 빠르고 정확한 '
 '멀티모달 이해·생성을 가능케 합니다.')
<class 'str'> Claude 모델의 학습 원리를 4 문장으로 한국어로 답변해 주세요.
('Claude는 대규모 언어 모델로, 인터넷의 방대한 텍스트를 바탕로 다음 토큰을 예측하도록 학습됩니다.  \n'
 '사람 트레이너가 대화 스타일, 안전, 유용성을 높이는 프롬프트-응답 쌍을 만들어 이를 미세조정합니다. 

##### 2) ChatPromptTemplate
* Tuple 형태의 system, user, assistant 메시지 지원
* 여러 개의 메시지를 조합하여 LLM에게 전달 가능
* 간결성과 가독성이 높고 단순한 구조

In [7]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    # role, message
    ("system", "This system is an expert in answering questions about {topic}. Please provide clear and detailed explanations."),
    ("human", "{model_name} 모델의 학습 원리를 설명해 주세요."),
])

messages = chat_prompt.format_messages(topic="AI", model_name="ChatGPT")
print(messages)

# 생성한 메시지를 바로 주입하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

print(type(response))
print(response.content)

[SystemMessage(content='This system is an expert in answering questions about AI. Please provide clear and detailed explanations.', additional_kwargs={}, response_metadata={}), HumanMessage(content='ChatGPT 모델의 학습 원리를 설명해 주세요.', additional_kwargs={}, response_metadata={})]
<class 'langchain_core.messages.ai.AIMessage'>
ChatGPT는 “GPT(Generative Pre-trained Transformer)” 아키텍처를 기반으로 하며, 크게 두 단계(Pre-training → Fine-tuning)를 거쳐 만들어집니다. 각 단계에서 수행하는 작업과 내부 원리를 단계별로 정리하면 다음과 같습니다.

----------------------------------------
1. Pre-training(사전 학습) – “언어 모델 만들기”
목표  
대량의 공개 문서(웹, 책, 위키 등)를 통해 “다음 토큰(token)이 무엇일까?”를 맞추는 일반적인 언어 모델을 학습합니다.  
→ 이 단계에서는 레이블(정답)이 필요 없는 자기지도 학습(self-supervised) 방식이므로, 방대한 양의 데이터를 저렴하게 활용할 수 있습니다.

내부 동작  
1) 토큰화  
   - 텍스트를 서브워드 단위(BPE/BBPE 등)로 자름  
2) 입력 임베딩  
   - 각 토큰은 고정 차원 벡터로 변환되고, 위치 정보도 더해져서 “입력 행렬”이 됨  
3) Transformer 블록 반복  
   - 멀티헤드 셀프 어텐션 → 피드포워드 네트워크 → 잔차 연결 & 층 정규화  
   - 어텐션은 문장 전체의 단어 간 관계(문법, 의미, 지식)를 학습  
4) 다음 토큰 예측  
   - 마지막 임베딩에 언어 모델링 헤드(선형 + 소프트

In [8]:
# 체인을 생성하여 호출하기
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

chain = chat_prompt | llm | StrOutputParser()

response = chain.invoke({"topic":"AI", "model_name":"ChatGPT"})
print(type(response))
print(response)

<class 'str'>
ChatGPT는 “GPT(Generative Pre-trained Transformer)”라는 대규모 언어 모델을 기반으로 하며,  
“이전 단어들이 주어졌을 때 다음 단어를 예측하는 확률 모델”을 학습하는 방식으로 동작합니다.  
전체 학습 과정은 크게 3단계로 나눌 수 있습니다.

----------------------------------------
1. Pre-training(사전 학습) – “많은 글을 읽고 세상 지식을 암기”
목표  
• 인터넷에 공개된 방대한 텍스트(수천억 토큰)를 바탕으로 “다음 토큰 예측”을 최적화  
• 결과적으로 세계 지식·문법·추론 패턴을 암기한 “기본 모델(Base)”을 얻음

기술적 핵심  
• Transformer 디코더(디코더-전용) 아키텍처  
• 입력 토큰 임베딩 + 위치 임베딩 → 다중-헤드 셀프 어텐션 → FFN 반복  
• 최종 softmax로 다음 토큰 확률 분포 출력  
• 손실 함수: 교차 엔트로피(Cross-Entropy)  
• 최적화: AdamW, LR 스케줄링(웜업 → cosine decay), bf16/FP16 혼합 정밀도  
• 확장 법칙: 파라미터 수↑, 데이터↑, 컴퓨트↑ → 테스트 손실↓ (Chinchilla 법칙 참고)

2. SFT(Supervised Fine-Tuning) – “지시문-응답 쌍을 학습”
목표  
• 사람이 작성한 “질문-답” 형식의 지시 데이터셋(1~10만 건)로 추가 학습  
• 모델이 “대화 처럼” 말하도록 톤과 형식을 조정

기술적 핵심  
• 동일한 “다음 토큰 예측” 손실을 사용하지만,  
  입력=지시문, 타깃=답변 형식으로 구성  
• 학습률 낮춰서 Pre-training 가중치 미세 조정(전이 학습)

3. RLHF(강화학습을 이용한 인간 피드백 학습) – “인간이 좋아하는 답변을 만들기”
3-1. 보상 모델(RM) 학습  
• 동일한 프롬프트에 대해 모델이 생성한 2개 이상의 답변을 사람이 선호도 순위 매김  
• RM =

#### 3) ChatPromptTemplate
* SystemMessagePromptTemplate와 HumanMessagePromptTemplate 클래스 사용
* 객체 지향적 접근 - Message 객체를 독립적으로 생성 가능
* 여러 조건에 따라 다른 시스템 메시지 선택

```python
if user_is_beginner:
    system_message = SystemMessagePromptTemplate.from_template("초보자를 위한 설명: {topic}")
else:
    system_message = SystemMessagePromptTemplate.from_template("전문가를 위한 상세 분석: {topic}")
```

In [9]:
# ChatMessagePromptTemplate 활용

from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    AIMessagePromptTemplate,
    ChatMessagePromptTemplate
)
from langchain_openai import ChatOpenAI

# 개별 메시지 템플릿 정의
system_message = SystemMessagePromptTemplate.from_template(
    "You are an AI expert in {topic}. Please provide clear and detailed explanations."
)
user_message = HumanMessagePromptTemplate.from_template(
    "{question}"
)
ai_message = AIMessagePromptTemplate.from_template(
    "This is an example answer about {topic}."
)

# ChatPromptTemplate로 메시지들을 묶기
chat_prompt = ChatPromptTemplate.from_messages([
    system_message,
    user_message,
    ai_message
])

# 메시지 생성
messages = chat_prompt.format_messages(topic="AI", question="What is deep learning?")

# LLM 호출
#llm = ChatOpenAI(model="gpt-3.5-turbo-0125")
response = llm.invoke(messages)

# 결과 출력
print(response.content)

#### ChatMessagePromptTemplate는 여러 종류의 메시지(시스템, 인간, AI)를 조합하여 복잡한 프롬프트를 생성할 때 유용합니다.
* SystemMessagePromptTemplate: 이 템플릿은 AI 모델에게 역할을 부여하거나 전반적인 규칙을 설정하는 시스템 메시지를 만듭니다. 위의 예시에서는 "번역을 도와주는 유용한 도우미"라는 역할을 지정합니다.
* HumanMessagePromptTemplate: 이 템플릿은 사용자의 질문이나 요청을 담는 인간 메시지를 만듭니다. 아래의 예시에서는 번역할 텍스트를 입력받습니다.
* ChatPromptTemplate.from_messages: 이 클래스 메서드는 시스템 메시지, 인간 메시지 등 여러 종류의 MessagePromptTemplate 객체들을 리스트로 받아 하나의 채팅 프롬프트 템플릿으로 통합합니다.
* format_messages: 이 메서드는 정의된 템플릿에 실제 값을 채워 넣어 [SystemMessage, HumanMessage] 형태의 리스트를 반환합니다. 이 리스트는 채팅 모델(Chat Model) 에 바로 전달될 수 있습니다.

In [10]:
# 필요한 라이브러리 임포트
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# 1. SystemMessagePromptTemplate와 HumanMessagePromptTemplate 생성
# SystemMessagePromptTemplate는 모델의 페르소나 또는 기본 지침을 설정합니다.
system_template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

# HumanMessagePromptTemplate는 사용자로부터 받는 입력 프롬프트를 정의합니다.
human_template = "{text_to_translate}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

# 2. ChatPromptTemplate 생성
# 위에서 만든 두 템플릿을 리스트로 묶어 ChatPromptTemplate을 만듭니다.
chat_prompt_template = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# 3. 프롬프트 포맷팅
# chat_prompt_template.format_messages()를 사용하여 최종 메시지 리스트를 생성합니다.
# 이 함수는 딕셔너리 형태의 입력 변수를 받습니다.
formatted_prompt = chat_prompt_template.format_messages(
    input_language="English",
    output_language="Korean",
    text_to_translate="I love programming."
)

# 4. 결과 출력
print(formatted_prompt)

# LLM 호출
response = llm.invoke(formatted_prompt)

# 결과 출력
print(response.content)

[SystemMessage(content='You are a helpful assistant that translates English to Korean.', additional_kwargs={}, response_metadata={}), HumanMessage(content='I love programming.', additional_kwargs={}, response_metadata={})]
나는 프로그래밍을 사랑해.


##### 4) FewShotPromptTemplate
* FewShotPromptTemplate은 모델이 특정 형식을 따르게 하거나, 일관된 응답을 생성하도록 유도할 때 유용합니다.
* 도메인 지식이 필요하거나, AI가 오답을 줄이고 더 신뢰할 만한 답변을 생성하도록 해야 할 때 효과적입니다.

##### 4-1) PromptTemplate을 사용하지 않는 경우

In [11]:
# PromptTemplate을 사용하지 않는 경우
from langchain_openai import ChatOpenAI

# model
#llm = ChatOpenAI(model="gpt-3.5-turbo")

# chain 실행
result = llm.invoke("태양계의 행성들을 간략히 정리해 주세요.")

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
태양계 행성 8개를 태양에서부터 간단 정리:

1. **수성**  
   가장 작고, 가까우며, 표면 온도 변화가 극심함.

2. **금성**  
   두꺼운 이산화탄소 대기로 온도가 높고, 표면 압력이 큼. 지구보다 태양에 더 가까워도 더 뜨거움.

3. **지구**  
   유일하게 생명이 존재하는 행성. 물이 액체 상태로 존재.

4. **화성**  
   붉은 색의 사막 행성. 옛날에는 물이 흘렀던 흔적이 있음.

5. **목성**  
   태양계에서 가장 큰 행성. 대기는 수소와 헬륨으로 이루어짐. 대형 폭풍(붉은 반점)이 있음.

6. **토성**  
   아름다운 고리가 특징. 밀도가 물보다 낮아 물 위에 떠 있을 수 있음.

7. **천왕성**  
   거의 옆으로 돌아가 있으며, 메탄이 포함된 대기로 인해 푸른색. 고리가 있음.

8. **해왕성**  
   태양계에서 가장 강한 바람이 분다. 푸른색이며, 가장 큰 위성 트리톤이 거꾸로 도는 역행 궤도에 있음.

요약:  
**안쪽 행성** (수성, 금성, 지구, 화성)는 암석형,  
**바깥 행성** (목성, 토성, 천왕성, 해왕성)은 가스형 또는 빙형.


##### 4-2) FewShotChatMessagePromptTemplate 사용하는 경우

In [12]:
# FewShotChatMessagePromptTemplate 사용하는 경우
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
from langchain_openai import ChatOpenAI

examples = [
    {
        "input": "뉴턴의 운동 법칙을 요약해 주세요.",
        "output": """### 뉴턴의 운동 법칙
1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.
2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.
3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다."""
    },
    {
        "input": "지구의 대기 구성 요소를 알려주세요.",
        "output": """### 지구 대기의 구성
- **질소 (78%)**: 대기의 대부분을 차지합니다.
- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.
- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.
- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다."""
    }
]

# 예제 프롬프트 템플릿
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

# FewShotChatMessagePromptTemplate 적용
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

# 최종 프롬프트 구성
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

# 모델 생성 및 체인 구성
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
chain = final_prompt | llm
print(chain)

# 테스트 실행
result = chain.invoke({"input": "양자컴퓨팅에 대하여 설명해 주세요."})
print(result.content)

first=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 초등학생도 쉽게 이해할 수 있도록 쉽게 설명하는 과학 교육자입니다.'), additional_kwargs={}), FewShotChatMessagePromptTemplate(examples=[{'input': '뉴턴의 운동 법칙을 요약해 주세요.', 'output': '### 뉴턴의 운동 법칙\n1. **관성의 법칙**: 힘이 작용하지 않으면 물체는 계속 같은 상태를 유지합니다.\n2. **가속도의 법칙**: 물체에 힘이 작용하면, 힘과 질량에 따라 가속도가 결정됩니다.\n3. **작용-반작용 법칙**: 모든 힘에는 크기가 같고 방향이 반대인 힘이 작용합니다.'}, {'input': '지구의 대기 구성 요소를 알려주세요.', 'output': '### 지구 대기의 구성\n- **질소 (78%)**: 대기의 대부분을 차지합니다.\n- **산소 (21%)**: 생명체가 호흡하는 데 필요합니다.\n- **아르곤 (0.93%)**: 반응성이 낮은 기체입니다.\n- **이산화탄소 (0.04%)**: 광합성 및 온실 효과에 중요한 역할을 합니다.'}], input_variables=[], input_types={}, partial_variables={}, example_prompt=ChatPromptTemplate(input_variables=['input', 'output'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[

#### 5-1) PartialPrompt 
* 프롬프트를 더 동적으로 활용할 수 있으며, AI 응답을 더 일관성 있게 조정 가능함

In [13]:
from datetime import datetime
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# 프롬프트 템플릿 정의 (부분 변수 적용)
prompt = PromptTemplate(
    template="{season}에 일어나는 대표적인 지구과학 현상은 {phenomenon}이 맞나요? {season}에 주로 발생하는 지구과학 현상을 3개 알려주세요",
    input_variables=["phenomenon"],  # 사용자 입력 필요
    partial_variables={"season": get_current_season()}  # 동적으로 계절 값 할당
)

# OpenAI 모델 초기화
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)

# 특정 계절의 현상 질의
query = prompt.format(phenomenon="태풍 발생")
result = llm.invoke(query)


# 결과 출력
print(f" 프롬프트: {query}")
print(f" 모델 응답: {result.content}")

 프롬프트: 가을에 일어나는 대표적인 지구과학 현상은 태풍 발생이 맞나요? 가을에 주로 발생하는 지구과학 현상을 3개 알려주세요
 모델 응답: 가을에 **태풍 발생**이 지구과학 현상으로 **맞긴 하지만**, 가을에 **"주로 발생한다"**고 보기는 어렵습니다. 태풍은 **여름~초가을(7~9월)**에 가장 많이 발생하지만, **계절에 국한된 현상은 아닙니다**. 즉, **가을에도 태풍이 오긴 하지만**, **가을에만 특별히 일어나는 대표적인 지구과학 현상은 아닙니다**.

---

### ✅ 가을에 **주로 발생하는** 대표적인 지구과학 현상 3가지:

1. **대기 안정화로 인한 맑은 날씨의 증가**  
   → 여름철 열대성 기단이 약해지고, 고위도의 차가운 기단이 내려오면서 **대기가 안정화**됩니다. 이로 인해 **가을하늘이 높고 맑아지는 현상**이 두드러집니다.

2. **기압골의 남하로 인한 기온 변화**  
   → 시베리아 고기압이 강해지면서 **기압골이 남하**하면서 **일교차가 크게 증가**하고, **급격한 기온 변화**가 나타납니다. 이는 **가을철 단풍**과도 밀접한 관련이 있습니다.

3. **황사 발생 가능성 감소 + 서풍 강화**  
   → 중국 내륙의 모래먼지가 동아시아로 오는 **황사**는 **봄철에 집중**되어 있고, **가을에는 서풍이 강화되면서 황사 발생 가능성이 줄어듭니다**. 이는 **계절풍의 변화**와 관련된 지구과학 현상입니다.

---

### 요약
- 태풍은 **가을에도 올 수 있지만**, **가을에만 특별히 발생하는 현상은 아닙니다**.
- 가을에 **특징적인 지구과학 현상**은  
  → **대기 안정화**,  
  → **기압골 남하로 인한 기온 변화**,  
  → **계절풍 변화로 인한 황사 감소**입니다.


In [14]:
from datetime import datetime
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 계절을 결정하는 함수 (남반구/북반구 고려)
def get_current_season(hemisphere="north"):
    month = datetime.now().month

    if hemisphere == "north":  # 북반구 (기본값)
        if 3 <= month <= 5:
            return "봄"
        elif 6 <= month <= 8:
            return "여름"
        elif 9 <= month <= 11:
            return "가을"
        else:
            return "겨울"
    else:  # 남반구 (계절 반대)
        if 3 <= month <= 5:
            return "가을"
        elif 6 <= month <= 8:
            return "겨울"
        elif 9 <= month <= 11:
            return "봄"
        else:
            return "여름"

# Step 1: 현재 계절 결정
season_name = get_current_season()  # 계절 값 얻기
print(f"현재 계절: {season_name}")

현재 계절: 가을


In [15]:

# Step 2: 해당 계절의 자연 현상 추천
prompt2 = ChatPromptTemplate.from_template(
    "{season}에 주로 발생하는 대표적인 지구과학 현상 3가지를 알려주세요. "
    "각 현상에 대해 간단한 설명을 포함해주세요."
)

# OpenAI 모델 사용
#llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.0)
##llm = ChatOpenAI(
##    #api_key=OPENAI_API_KEY,
##    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
##    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
##    temperature=0.0
##)

# 체인 2: 자연 현상 추천 (입력: 계절 → 출력: 자연 현상 목록)
chain2 = (
    {"season": lambda x : season_name}  # chain1의 출력을 season 변수로 전달
    | prompt2
    | llm
    | StrOutputParser()
)

# 실행: 현재 계절에 따른 자연 현상 추천
response = chain2.invoke({})
print(f"\n {season_name}에 발생하는 자연 현상:\n{response}")

KeyboardInterrupt: 

#### 5-2) PartialPrompt 
* API 호출 데이터, 시간 정보, 사용자 정보 등을 반영할 때 매우 유용함

In [ ]:
import requests
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# 실시간 환율을 가져오는 함수
def get_exchange_rate():
    response = requests.get("https://api.exchangerate-api.com/v4/latest/USD")
    data = response.json()
    return f"1달러 = {data['rates']['KRW']}원"

# Partial Prompt 활용
prompt = PromptTemplate(
    template="현재 {info} 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.",
    input_variables=[],  # 사용자 입력 없음
    partial_variables={"info": get_exchange_rate()}  # API에서 가져온 데이터 자동 반영
)

# LLM 모델 설정
#llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.0)

# 모델에 프롬프트 전달 및 응답 받기
response = llm.invoke(prompt.format())

# 결과 출력
print(" 프롬프트:", prompt.format())
print(" 모델 응답:", response.content)

 프롬프트: 현재 1달러 = 1377.98원 기준으로 환율 정보를 알려드립니다. 이에 대한 분석을 제공해 주세요.
 모델 응답: ## 2024년 5월 8일 환율 분석

2024년 5월 8일, 1달러는 1377.98원으로 환산됩니다. 이는 2024년 5월 8일 한국은행이 고시한 환율에 따른 정보이며, 실제 거래 시 소매환율(환전 시 적용되는 환율)이 적용됩니다.

* 전일 환율: 1달러 = 1376.7원 
* 상승/하락: 상승 (전일 대비 1.28원 상승) 
* 거래량: (해당 정보는 실시간으로 제공되지 않으므로, 별도로 확인 필요) 

## 환율에 영향을 주는 요인들

1. **경제 지표**: 미국의 비농업 부문 고용 지표, GDP 성장률, 인플레이션율 등 경제 지표는 환율에 큰 영향을 미칩니다. 
2. **글로벌 경제 상황**: 세계 경제의 불확실성, 무역 전쟁, 국제 유가 변동 등도 환율에 영향을 줍니다. 
3. **통화 정책**: 미국 연방준비제도(Fed)의 금리 결정과 한국은행의 통화 정책도 환율 변동에 영향을 미치는 중요한 요인입니다. 
4. **정치적 요인**: 정치적 불확실성이나 지정학적 리스크도 환율에 영향을 줄 수 있습니다.

## 최근 환율 동향

2024년 5월 8일 기준, 원화 가치는 미국 달러 대비 소폭 상승한 것으로 나타납니다. (전일 대비 1.28원 상승한 1377.98원) 

이는 최근 글로벌 경제의 불확실성이 다소 완화되고, 한국 경제에 대한 긍정적인 전망이 반영된 결과일 수 있습니다.

하지만, 환율은 다양한 요인에 의해 변동성이 크므로, 지속적인 모니터링이 필요합니다. 특히, 미국의 통화 정책과 글로벌 경제 상황의 변화는 원화 가치에 큰 영향을 미칠 것입니다.

## 전망

향후 환율 전망은 여러 가지 요인에 따라 달라질 수 있습니다. 
* 미국의 금리 인하 가능성
* 한국의 경제 성장률
* 글로벌 무역 상황

등을 고려해야 합니다.

## 요약

2024년 5월 8일, 1달러는 1377.98원으로 환산되며, 전일 대비 소폭 